In [1]:
import pandas as pd
import numpy as np

def calcola_densita_acqua(temperatura):
    """Calcola la densità dell'acqua in funzione della temperatura."""
    return 1000 - 0.097 * (temperatura - 4)**2

def calcola_variazione_livello_teorica(pressione_iniziale, pressione_finale, densita):
    """Calcola la variazione del livello idrometrico teorica dovuta alla pressione."""
    g = 9.81  
    delta_p = (pressione_finale - pressione_iniziale) * 100  # conversione da hPa a Pa
    return -delta_p / (densita * g)

def calcola_incidenza_percentuale(df, colonna_target, colonna_fattore):
    """Calcola l'incidenza percentuale di un fattore sulla variabile target."""
    # Calcola R² (coefficiente di determinazione)
    correlation = df[colonna_target].corr(df[colonna_fattore])
    r_squared = correlation ** 2
    
    incidenza = r_squared * 100
    return incidenza

def calcola_dati():
    file_name = "BARI_24.csv"
    df = pd.read_csv(file_name, delimiter=";")
    
    # Conversione e pulizia dei dati
    for colonna in ['LIVELLO IDROMETRICO', 'PRESSIONE ATMOSFERICA', 'TEMPERATURA ARIA']:
        df[colonna] = df[colonna].astype(str).str.replace(',', '.')
        df[colonna] = pd.to_numeric(df[colonna], errors='coerce')
    
    # Gestione date e ore
    df['ORA'] = pd.to_datetime(df['ORA'], format='%H:%M').dt.time
    df['DATA_ORA'] = pd.to_datetime(df['DATA'] + ' ' + df['ORA'].astype(str))
    
    # Calcolo medie orarie
    df['LIVELLO_IDROMETRICO_MEDIA ORARIA'] = df['LIVELLO IDROMETRICO'].rolling(window=6, min_periods=6).mean()
    df['TEMPERATURA_ARIA_MEDIA_ORARIA'] = df['TEMPERATURA ARIA'].rolling(window=6, min_periods=6).mean()
    
    df.loc[df.index >= 4, 'LIVELLO_IDROMETRICO_MEDIA ORARIA'] = df['LIVELLO IDROMETRICO'].rolling(window=6).mean().iloc[4:].values
    
    # Filtraggio dati orari
    df_ora = df[df['DATA_ORA'].dt.minute == 0].reset_index(drop=True)
    df_ora.index += 1
    df_ora = df_ora[df_ora['DATA_ORA'].dt.time >= pd.to_datetime("00:00", format='%H:%M').time()]
    df_ora['ORA'] = df_ora['DATA_ORA'].dt.strftime('%H:%M')
    
    # Calcolo degli effetti fisici
    df_ora['DENSITA_ACQUA'] = df_ora['TEMPERATURA_ARIA_MEDIA_ORARIA'].apply(calcola_densita_acqua)
    
    # Calcolo variazioni
    df_ora['VARIAZIONE_LIVELLO_TEORICA'] = 0.0
    for i in range(1, len(df_ora)):
        df_ora.loc[df_ora.index[i], 'VARIAZIONE_LIVELLO_TEORICA'] = calcola_variazione_livello_teorica(
            df_ora['PRESSIONE ATMOSFERICA'].iloc[i-1],
            df_ora['PRESSIONE ATMOSFERICA'].iloc[i],
            df_ora['DENSITA_ACQUA'].iloc[i]
        )
    
    df_ora['VARIAZIONE_LIVELLO_EFFETTIVA'] = df_ora['LIVELLO_IDROMETRICO_MEDIA ORARIA'].diff()
    
    return df_ora

def analizza_risultati(df):
    """Analizza e stampa i risultati dell'analisi."""
    print("Analisi dei fattori atmosferici sul livello idrometrico a Bari")
    print("-" * 60)
    
    # Calcolo incidenze percentuali
    incidenza_temperatura = calcola_incidenza_percentuale(
        df, 'LIVELLO_IDROMETRICO_MEDIA ORARIA', 'TEMPERATURA_ARIA_MEDIA_ORARIA'
    )
    
    incidenza_pressione = calcola_incidenza_percentuale(
        df, 'LIVELLO_IDROMETRICO_MEDIA ORARIA', 'PRESSIONE ATMOSFERICA'
    )
    
    # Statistiche descrittive
    print("\nStatistiche descrittive:")
    print(f"Temperatura media: {df['TEMPERATURA_ARIA_MEDIA_ORARIA'].mean():.2f}°C")
    print(f"Pressione media: {df['PRESSIONE ATMOSFERICA'].mean():.2f} hPa")
    print(f"Livello idrometrico medio: {df['LIVELLO_IDROMETRICO_MEDIA ORARIA'].mean():.2f} m")
    
    # Stampa incidenze
    print("\nIncidenze percentuali sul livello idrometrico:")
    print(f"Incidenza della temperatura: {incidenza_temperatura:.2f}%")
    print(f"Incidenza della pressione: {incidenza_pressione:.2f}%")
    
    # Calcolo dell'incidenza di altri fattori non misurati
    incidenza_altri_fattori = 100 - (incidenza_temperatura + incidenza_pressione)
    print(f"Incidenza di altri fattori: {incidenza_altri_fattori:.2f}%")
    
    return {
        'incidenza_temperatura': incidenza_temperatura,
        'incidenza_pressione': incidenza_pressione,
        'incidenza_altri_fattori': incidenza_altri_fattori,
        'dataframe': df
    }

# Esecuzione dell'analisi
dati_bari = calcola_dati()
risultati = analizza_risultati(dati_bari[["ORA", "LIVELLO_IDROMETRICO_MEDIA ORARIA", 
                                        "TEMPERATURA_ARIA_MEDIA_ORARIA", "PRESSIONE ATMOSFERICA",
                                        "VARIAZIONE_LIVELLO_TEORICA", "VARIAZIONE_LIVELLO_EFFETTIVA"]])

Analisi dei fattori atmosferici sul livello idrometrico a Bari
------------------------------------------------------------

Statistiche descrittive:
Temperatura media: 11.77°C
Pressione media: 1015.96 hPa
Livello idrometrico medio: -0.17 m

Incidenze percentuali sul livello idrometrico:
Incidenza della temperatura: 7.69%
Incidenza della pressione: 31.53%
Incidenza di altri fattori: 60.78%
